# 原始数据加载（docs）

In [ ]:
# 版本控制
version = 'V3'

任务说明（以文献数据处理为例）：
- 1.将EXCEL数据转换为json格式，按照3TI 、1AB和2KEY WORDS，完成文本的字符串拼接，json文件中需要`year`、`title`、`abstract`、`keywords`以及`combined_text`字段；（AI应该很容易写出来相应的转换代码，该文件作为原始主题建模的原始数据文件）；
- 2.变量`docs`是模型的输入，将json文件中的`combined_text`传入到该向量中形成一个列表型数据；
- 3.根据后续代码跑通这一套BERTopic技术路线

In [ ]:
import json
# 从JSON文件加载数据
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 打印前两条数据进行检查
for item in data[-2:]:
    print(item['year'])
    #print(item['title'])
    #print(item['abstract'])
    print(item['combined_text'])

# 文本
docs = [item['combined_text'] for item in data]

In [ ]:
print('共获取数据总量:', len(docs),'条')

# 创建模型

## 最推荐的英文嵌入模型
EMBEDDING_MODELS = {
    # 🔥 强烈推荐：专门为句子相似度训练
    'all-MiniLM-L6-v2': {
        'name': 'all-MiniLM-L6-v2',
        'description': '轻量高效，平衡速度和质量',
        'size': '80MB',
        'dimension': 384
    },
    
    'all-mpnet-base-v2': {
        'name': 'all-mpnet-base-v2',
        'description': '效果最好，SOTA质量',
        'size': '420MB',
        'dimension': 768
    },
    
    'paraphrase-MiniLM-L6-v2': {
        'name': 'paraphrase-MiniLM-L6-v2',
        'description': '擅长语义相似度',
        'size': '80MB',
        'dimension': 384
    },
    
    # 通用的BERT模型
    'bert-base-nli-mean-tokens': {
        'name': 'bert-base-nli-mean-tokens',
        'description': '自然语言推理任务训练',
        'size': '440MB',
        'dimension': 768
    },
    
    # 科学/学术专用
    'all-MiniLM-L12-v2': {
        'name': 'all-MiniLM-L12-v2',
        'description': '12层版本，更深的表示',
        'size': '120MB',
        'dimension': 384
    },
    
    # 多语言（如果包含其他语言）
    'paraphrase-multilingual-MiniLM-L12-v2': {
        'name': 'paraphrase-multilingual-MiniLM-L12-v2',
        'description': '多语言支持',
        'size': '470MB',
        'dimension': 384
    }
}

In [ ]:
import os
# 1. 设置国内镜像源
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from bertopic import BERTopic
import torch # 新增：导入PyTorch（sentence_transformers的底层）
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [ ]:
# --- 核心：检查并指定设备 ---
# 检查CUDA（即NVIDIA GPU支持）是否可用
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU可用，正在使用: {torch.cuda.get_device_name(0)}")
    # 可选：如果你想使用特定的GPU（如第0块）
    # device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
    print("⚠️  未检测到可用GPU，将使用CPU。")

In [ ]:
import torch
print(torch.__version__) # 查看PyTorch版本
print(torch.cuda.is_available()) # 核心：查看CUDA是否可用

In [ ]:
# from sentence_transformers import SentenceTransformer
# import os

# # 1. 指定你希望保存模型的本地目录
# local_model_path = "./my_models/all-mpnet-base-v2"

# # 2. 检查本地是否已有模型，没有则下载并保存
# if not os.path.exists(local_model_path):
#     print("模型不存在，正在下载并保存到本地...")
#     model = SentenceTransformer('all-mpnet-base-v2')
#     embedding_model.save(local_model_path)  # 关键：保存到指定路径
#     print(f"模型已保存至：{local_model_path}")
# else:
#     print("模型已存在，从本地加载...")

In [ ]:
# 3. 之后都从这个路径加载，完全无需网络
embedding_model = SentenceTransformer(f"./my_models/all-mpnet-base-{version}")

In [ ]:
# 2. 计算并保存词向量
def compute_and_save_embeddings(docs, save_path='embeddings_test'):
    """计算并保存词向量"""
    if os.path.exists(save_path):
        print(f"加载已保存的嵌入: {save_path}")
        with open(save_path, 'rb') as f:
            embeddings = pickle.load(f)
    else:
        print("计算新的嵌入...")
        embeddings = embedding_model.encode(docs, show_progress_bar=True)
        with open(save_path, 'wb') as f:
            pickle.dump(embeddings, f)
        print(f"嵌入已保存到: {save_path}")
    return embeddings

In [ ]:
import json

def load_stopwords(file_path):
    """从JSON文件加载停用词列表"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            stopwords = json.load(f)
        print(f"已加载 {len(stopwords['common'])} 个停用词")
        return stopwords['common']  # 转换为集合提高查找效率
    except FileNotFoundError:
        print(f"错误：文件 {file_path} 不存在")
        return set()
    except json.JSONDecodeError:
        print(f"错误：文件 {file_path} 不是有效的JSON格式")
        return set()

# # 测试示例
# stopwords = load_stopwords('data\stopwords.json')
# print(stopwords)

# UMAP（降维模型）和HDBSCAN（聚类模型）参数设置

In [ ]:
# # 2. 创建UMAP降维模型
# umap_model = UMAP(
#   n_neighbors=20, # 控制局部与全局结构的平衡,决定每个点要考虑多少个最近邻居,值越大（如15-200）：更关注全局结构，获得更宏观的视图;值越小（如5-15）：更关注局部结构，捕捉更多细节。
#   n_components=5, # 降维后的维度，通常设置为2或3以便可视化，但BERTopic中建议设置为5以保留更多信息
#   min_dist=0.0, # 控制嵌入空间中点之间的最小距离,值越小（如0.0-0.1）：嵌入空间更紧凑，点之间距离更近;值越大（如0.5-0.99）：嵌入空间更稀疏，点之间距离更远。
#   metric='cosine', # 距离度量方法，常用的有'cosine'（余弦相似度）和'euclidean'（欧几里得距离）,文本数据、高维稀疏数据、TF-IDF向量等通常使用'cosine'，而低维密集数据通常使用'euclidean'
#   random_state=5  # 设置随机种子,确保结果可重复性,UMAP有随机初始化过程，设置此参数保证每次运行结果一致
# )

# # 3. 创建HDBSCAN聚类模型
# # 如果要建设离群值，可以减小下面两个参数min_cluster_size min_samples
# # https://hdbscan.readthedocs.io/en/latest/faq.html
# hdbscan_model = HDBSCAN(
#   min_cluster_size=50, # 最小聚类大小,决定了形成一个聚类所需的最小数据点数量。较大的值会导致更少但更稳定的聚类，而较小的值会产生更多但可能不太稳定的聚类。
# # 过滤掉太小的噪声点聚集
# # 控制聚类粒度：值越大，生成的聚类越少、越稳健
# # 这是HDBSCAN中最重要的参数，对结果影响最大

#   min_samples=3, # 最小样本数,影响聚类的核心点定义。较高的值会使得聚类更严格，可能导致更多的离群点，而较低的值则更宽松，可能形成更多的聚类。
#   metric='euclidean'
# )

In [ ]:
# 1. 定义参数配置中心
model_config = {
    "UMAP": {
        "n_neighbors": 20,
        "n_components": 5,
        "min_dist": 0.0,
        "metric": 'cosine',
        "random_state": 5
    },
    "HDBSCAN": {
        "min_cluster_size": 50,
        "min_samples": 3,
        "metric": 'euclidean',
        "prediction_data": True  # 建议开启，以便后续进行维度转换
    }
}

# 2. 方便打印数值的函数
def print_config(config):
    print("="*30)
    print(" 当前模型超参数配置 ")
    print("="*30)
    for model_name, params in config.items():
        print(f"\n[{model_name} 参数]:")
        for key, value in params.items():
            # 使用 ljust 让输出对齐，更易读
            print(f"  - {key.ljust(20)}: {value}")
    print("="*30)

# 执行打印
print_config(model_config)

# 3. 实例化模型
# 使用 ** 直接解包字典参数
umap_model = UMAP(**model_config["UMAP"])
hdbscan_model = HDBSCAN(**model_config["HDBSCAN"])

print("\n模型已成功根据上述参数初始化。")

# BERTopic模型创建

In [ ]:
# 停用词地址
stop_words = load_stopwords('data\stopwords.json')
# 创建CountVectorizer模型,自定义停用词
vectorizer_model = CountVectorizer(stop_words=stop_words)

# 正式创建BERTopic模型
topic_model = BERTopic(
  embedding_model=embedding_model,
  vectorizer_model=vectorizer_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
)

# 训练模型

In [ ]:
# 2. 计算并保存词向量
embeddings = compute_and_save_embeddings(docs, 'results\embedding_results\embeddings_patents_zf_v1.pkl')

In [ ]:
# 你的代码流程：
# +-------------------+       +-------------------------+       +---------------+
# | 你的预计算嵌入     |------>| BERTopic.fit_transform()|------>|最终主题结果   |
# | embeddings_patents|       |                         |       |               |
# | _zf_v1.pkl        |       |                         |       | topics, probs |
# +-------------------+       +-------------------------+       +---------------+
#                                    │
#                                    ▼ 内部处理流程
#                         +-----------------------+
#                         │ 1. 接收你的 embeddings │
#                         +-----------------------+
#                                    │
#                                    ▼
#                         +-----------------------+
#                         │ 2. ✅ UMAP 降维        │← 这里使用 UMAP！
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +-----------------------+
#                         │ 3. HDBSCAN 聚类        │
#                         +-----------------------+
#                                    │
#                                    ↓
#                         +----------------------------------+
#                         │ 4. Vectorizer\c-TF-IDF提取主题词等│
#                         +----------------------------------+
topics, probs = topic_model.fit_transform(docs, embeddings=embeddings) #传入训练好的词向量
# 查看主题
topic_model.get_topic_info()

# 保存聚类结果

In [ ]:
# # 保存整个主题模型
# topic_model.save("bertopic_patents_zf_amb_v3")
# print("主题模型已保存到 bertopic_patents_zf_amb_v3")

## 直接加载初始模型
topic_model = BERTopic.load(f"bertopic_patents_zf_amb{version}")
print("初始主题模型加载成功！")

In [ ]:
topic_docs = topic_model.get_document_info(docs)
with open(r'data/patents_zf.json', 'r', encoding='utf-8') as file:
  data = json.load(file)
  # 假设JSON是字典列表
abstract = [item['abstract'] for item in data]  # 提取文本
years = [item['year'] for item in data]  # 提取年份
title = [item['title'] for item in data]  # 提取标题
#author_keywords = [item['author_keywords'] for item in data]  # 提取关键词
# 插入到聚类结果

topic_docs.insert(1, '标题', title)
topic_docs.insert(2, '年份', years)
topic_docs.insert(3, '摘要', abstract)
#topic_docs.insert(4, '作者关键词', author_keywords)
topic_docs.to_csv('./patents_zf_Clustering_detailed_result_V3.csv')

# 原始主题可视化

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.visualize_topics()

In [ ]:
reduced_embeddings = UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

# 层次聚类

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(docs)
hierarchical_clustering = topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
from bertopic import BERTopic
import plotly.io as pio

# 创建交互式层次树
hierarchical_fig = topic_model.visualize_hierarchy(
    hierarchical_topics=hierarchical_topics,
    width=1000,
    height=800
)

# 显示图形
hierarchical_fig.show()

# 保存为HTML以便交互
hierarchical_fig.write_html(f"hierarchical_topics_{version}.html")
print(f'已经创建层次树，并保存至hierarchical_topics_{version}.html')

# 计算可以进一步合并的主题

In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import fcluster, cut_tree

# 1. 针对数据结构提取 Scipy 链接矩阵
def get_linkage_matrix_final(df):
    # 提取左子节点、右子节点、距离
    # 你的列名是 Child_Left_ID, Child_Right_ID, Distance
    left = df['Child_Left_ID'].values.astype(float)
    right = df['Child_Right_ID'].values.astype(float)
    dist = df['Distance'].values.astype(float)
    
    # 提取主题数量：由于你的 Topics 列存的是列表，我们需要计算每个列表的长度
    num_topics = df['Topics'].apply(len).values.astype(float)
    
    # 合并为 Scipy 需要的 Linkage Matrix (n-1, 4)
    return np.column_stack([left, right, dist, num_topics])

# 2. 执行矩阵转换
# 注意：Scipy 要求矩阵按距离从小到大排序，BERTopic 返回的顺序通常是反的（根节点在最前）
df_sorted = hierarchical_topics.sort_values('Distance')
linkage_matrix = get_linkage_matrix_final(df_sorted)

# 3. 进行分组
# n_groups 是你想要的分组数量，例如 20 组------------------------------------------------------
n_groups = 20 
clusters = cut_tree(linkage_matrix, n_clusters=n_groups).flatten()

# 4. 映射回原始主题标签
# 获取所有原始主题 ID（排好序，确保与 cut_tree 结果对应）
all_topics = sorted([t for t in topic_model.get_topics().keys() if t != -1])

df_res = pd.DataFrame({
    "topic_id": all_topics,
    "group": clusters
})

# 5. 加上语义标签方便查看
topic_labels = topic_model.generate_topic_labels(nr_words=3, separator="_")
id_to_label = {int(l.split("_")[0]): l for l in topic_labels if not l.startswith("-1")}
df_res['label'] = df_res['topic_id'].map(id_to_label)

# 查看最终结果
print(df_res.head(20))

# 检查每个组包含的主题数
print("\n每组主题数量分布：")
print(df_res['group'].value_counts().sort_index())

In [ ]:
# prompt = f"""
# 请按照国防科技专家角色，将以下关键词列表转化为 JSON 格式的主题映射。
# 要求输出格式：
# {{
#     0: "主题名称",
#     1: "主题名称"
# }}

# 待处理数据：
# {input_keywords}
# """

In [ ]:
# 1. 按照 group 分组，并将 topic_id 聚合为列表
grouped_topics = df_res.groupby('group')['topic_id'].apply(list).reset_index()

# 2. 格式化输出
print(f"共识别出 {len(grouped_topics)} 个主题组：\n")

for _, row in grouped_topics.iterrows():
    # 将 group 编号加 1 方便阅读（从第一组开始计数）
    group_num = row['group'] + 1
    topic_list = row['topic_id']
    
    # 打印格式：第一组：[0, 1, 2, ...]
    print(f"第{group_num}组：{topic_list}")

In [ ]:
# 3. 如果你想同时看到这组主题的关键词标签（方便验证语义一致性）
print("\n" + "="*30 + "\n详细语义分组：\n")
for _, row in grouped_topics.iterrows():
    group_num = row['group'] + 1
    # 获取该组下所有主题的标签
    labels = df_res[df_res['group'] == row['group']]['label'].tolist()
    print(f"第{group_num}组 包含主题：")
    for lbl in labels:
        print(f"  - {lbl}")
    print("-" * 20)

In [ ]:
# 1. 提取嵌套列表格式：[[组1序号], [组2序号], ...]
nested_topic_lists = grouped_topics['topic_id'].tolist()

print("--- 嵌套列表输出 ---")
print(nested_topic_lists)
print("\n" + "="*50 + "\n")

In [ ]:
# 2. 合并后的主题理解方法一：提取每组的代表性关键词（丢失c-tf-idf权重）
# 逻辑：获取每组内所有主题的关键词，计算词频，取前 10 个作为组关键词
group_keywords = {}

for _, row in grouped_topics.iterrows():
    group_num = row['group']
    topics_in_group = row['topic_id']
    
    all_words_in_group = []
    for t_id in topics_in_group:
        # 获取单个主题的关键词 (返回的是 [(word, score), ...])
        words = [word for word, _ in topic_model.get_topic(t_id)]
        all_words_in_group.extend(words)
    
    # 统计词频并取前 10 个最能代表本组的词
    from collections import Counter
    # 过滤掉一些极其常见的重复词，保留每组独特的语义
    most_common_words = [word for word, count in Counter(all_words_in_group).most_common(10)]
    group_keywords[group_num + 1] = most_common_words

# 3. 格式化展示组关键词
print("--- 各组代表性关键词 ---")
for g_id, keywords in group_keywords.items():
    print(f"第{g_id}组共同特征词: {', '.join(keywords)}")

# 合并主题

In [ ]:
print(len(nested_topic_lists))

In [ ]:
# topic_model.merge_topics(docs, [
#     [9,10,7,13,37,33,29,41,20,22],
#     [50,6,38,19,14,16,54],
#     [18,51,40,28,52,31,24],
#     [3,23,47],
#     [36,2,39,5,42,11,15,17,0,21,27,25,48,53,34,4,30,46,45,12,44,55],
#     [49,56,1,8,32,26],
#     [43,35]
# ])

# 合并主题并显示
topic_model.merge_topics(docs, nested_topic_lists)
topic_model.get_topic_info()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_docs = topic_model.get_document_info(docs)
topic_docs.to_csv(f'./patents_zf_merged20聚类结果{version}.csv')

In [ ]:
reduced_embeddings = UMAP(n_neighbors=20, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings, hide_document_hover=True)

In [ ]:
# 获取所有主题的字典 {topic_id: [(word, score), ...]}
all_topics_dict = topic_model.get_topics()

print("索引：关键词名称")
# 遍历字典并排序
for topic_id in sorted(all_topics_dict.keys()):
    if topic_id != -1:  # 排除离群值主题
        # 提取单词并用逗号连接
        words = [word for word, score in all_topics_dict[topic_id]]
        words_str = ", ".join(words)
        print(f"{topic_id}：{words_str}")

In [ ]:
# 准备自定义标题列表
custom_titles = [f"标题_{i}: {doc}" for i, doc in enumerate(docs)]
# 使用自定义标签（需要先设置）
topic_model.set_topic_labels({
    0: "1.武器装备结构设计与制图", 
    1: "2.半导体工艺与微电子器件", 
    2: "3.军事信息系统与数据安全",
    3: "4.能源动力与先进推进系统",
    4: "5.生物防御与基因靶向技术",
    5: "6.战场医疗与官兵健康监测",
    6: "7.高性能含能材料与化学防御",
    7: "8.定向能武器与激光光电对抗",
    8: "9.微纳流控与生化传感侦察",
    9: "10.疫苗研发与病毒载体防御",
    10: "11.无人系统感知与模拟仿真",
    11: "12.野外战术电源与能量管理",
    12: "13.航空航天复合材料与隐身涂层",
    13: "14.磁特性监测与战略核磁技术",
    14: "15.热能循环与航空燃油管理",
    15: "16.雷达侦察与电子战对抗",
    16: "17.脑控技术与神经通信接口",
    17: "18.核生化探测与辐射成像成像",
    18: "19.航空发动机与涡轮叶片技术",
    19: "20.战伤修复与特种生物材料"
})

# 在可视化中传递自定义标题
fig = topic_model.visualize_documents(
    docs=docs,
    reduced_embeddings=reduced_embeddings,
    custom_labels=custom_titles,  # 使用自定义标签
    hide_document_hover=True
)

In [ ]:
# 更新字体样式
fig.update_layout(
    title="GF主题分布情况",
    # 居中标题
    title_x=0.5,
    font=dict(
        family="KaiTi",  # 中文字体
        size=20,
        color="black",
        weight="bold"  # 设置字体粗细
    ),
    title_font=dict(
        family="KaiTi",
        size=40,
        color="black",
        weight="bold"  # 设置字体粗细
    )
)

fig.show()

In [ ]:
# topics_in_group 主题数
# n_words 代表词数
# height/width 每个子图的高/宽
fig = topic_model.visualize_barchart(top_n_topics=20,n_words=10,height=500, width = 300,custom_labels=custom_titles) # 也可以设置 n_words 指定词数
fig.show()
fig.write_html(f'topic_words_merged_scores{version}.html')

# 时序主题模型

In [ ]:
# # 使用自定义标签（需要先设置）
# topic_model.set_topic_labels({
#     0: "基础信息检索理论与方法", 
#     1: "人工智能与学术信息应用", 
#     2: "网络安全与隐私增强检索",
#     3: "领域知识组织与语义检索",
#     4: "多模态信息检索",
#     5: "问答系统与知识推理",
#     6: "量子与物理启发的检索模型",
# })
# import pandas as pd

# 读取CSV文件
#df = pd.read_csv('patents_zf_Clustering_result_V2.csv')  # 如果文件是CSV格式
# 或者如果文件是其他格式，如xlsx：
# df = pd.read_excel('./data/时间.xlsx')
with open('data\patents_zf.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# 打印前两条数据进行检查
for item in data[-2:]:
    print(item['year'])
    #print(item['title'])
    #print(item['abstract'])
    print(item['combined_text'])

# 文本
timestamps = [item['year'] for item in data]

# 提取year列并转换为列表
#timestamps = df[''].astype(int).tolist()

print(len(timestamps), timestamps[:10])

In [ ]:
topics_over_time = topic_model.topics_over_time(docs, timestamps, global_tuning=False, evolution_tuning=False)
fig_topic_time = topic_model.visualize_topics_over_time(topics_over_time,custom_labels=custom_titles)
fig_topic_time.write_html(f'topic_overtime_merged{version}.html')